In [99]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, validation_curve, learning_curve
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import pandas as pd
import joblib
import json
import pickle

In [100]:
# Load the transformers
loaded_power_transformer = joblib.load('power_transformer.joblib')
loaded_standard_scaler = joblib.load('standard_scaler.joblib')
skewed_columns = joblib.load('skewed_columns.joblib')
selected_features_b_pca = joblib.load('X_selected_features.joblib')
best_model = joblib.load('best_model.joblib')
pca = joblib.load('pca_model.joblib')
selected_features_a_pca = joblib.load('selected_features.joblib')




In [ ]:
data = pd.read_excel('FinalTestDataset2024.xls')
data.head()

,ID,Age,ER,PgR,HER2,TrippleNegative,ChemoGrade,Proliferation,HistologyType,LNStatus,...,original_glszm_SmallAreaHighGrayLevelEmphasis,original_glszm_SmallAreaLowGrayLevelEmphasis,original_glszm_ZoneEntropy,original_glszm_ZonePercentage,original_glszm_ZoneVariance,original_ngtdm_Busyness,original_ngtdm_Coarseness,original_ngtdm_Complexity,original_ngtdm_Contrast,original_ngtdm_Strength
0,TRG002219,47.0,1,1,0,0,3,2,1,1,...,0.498350,0.498350,3.144594,0.003447,8257693.277,150.048587,0.001753,0.037110,0.001369,0.001513
1,TRG002222,41.0,1,1,0,0,3,2,1,0,...,0.622381,0.622381,2.061654,0.006535,1568441.643,26.484938,0.009649,0.019352,0.000321,0.008285
2,TRG002223,53.0,0,0,0,1,2,1,1,1,...,0.412482,0.412482,3.440353,0.005391,2656924.827,174.606929,0.001594,0.075152,0.005255,0.001444
3,TRG002235,46.0,1,1,0,0,2,1,1,1,...,0.378333,0.378333,3.531715,0.007102,1714787.173,96.787378,0.002772,0.053377,0.002666,0.002406
4,TRG002240,39.0,0,0,1,0,2,2,1,1,...,0.524767,0.524767,2.186214,0.007896,510479.346,12.789071,0.020072,0.023140,0.000463,0.017172


In [102]:
data.drop('ID', inplace=True, axis=1)
data.replace(999, np.nan, inplace=True)


for column in data:  
            data[column].fillna(data[column].median())


In [103]:


def set_outliers_to_z_score_limit(series): 
        # Exclude target features
        threshold_upper = 3 
        threshold_lower = -3
        
        mean = series.mean()
        std = series.std()
        
        replacement_upper_limit = (3 * std) + mean
        replacement_lower_limit = (-3 * std) + mean
        
        outlier_count= 0 
        for index, value in series.items():
            z_test = (value - mean) / std 
            
            if z_test > threshold_upper: 
                series[index] = replacement_upper_limit
                outlier_count+=1
                
            elif z_test < threshold_lower:
                series[index] = replacement_lower_limit
                outlier_count+=1
            
        #if outlier_count > 0:
            #print(f'column {series.name} had {outlier_count} outliers')
                    
        return series

# Discrete data is not affected by this function, so all features can be run through. 
data.apply(set_outliers_to_z_score_limit, axis=0)
data.reset_index(drop=True, inplace=True)

numerical_columns = data.columns[data.nunique() > 10].tolist()

# Apply Yeo-Johnson Transformation
data[skewed_columns] = loaded_power_transformer.transform(data[skewed_columns])

# Step 4: Check skewness again
skewness_after = data[numerical_columns].skew()
print("\nSkewness after transformation:")
print(skewness_after)

#drop outcomes and discrete data
dropped_features = ['ER', 'PgR', 'HER2', 'TrippleNegative', 'ChemoGrade', 'Proliferation', 'HistologyType', 'LNStatus', 'TumourStage', 'Gene']
continous_data = data.drop(columns=dropped_features, axis=1)

# Apply StandardScaler only to continuous variables

continuous_scaled = loaded_standard_scaler.transform(continous_data)

# Recombine data
continuous_scaled_df = pd.DataFrame(continuous_scaled, columns=continous_data.columns)
discrete_data = data[dropped_features]
scaled_data = pd.concat([discrete_data.reset_index(drop=True), continuous_scaled_df.reset_index(drop=True)], axis=1)

# One-hot encode the categorical variables
discrete_features = ['ER', 'PgR', 'HER2', 'TrippleNegative',
                     'ChemoGrade', 'Proliferation', 'HistologyType', 
                     'LNStatus', 'TumourStage', 'Gene']
 

preprocessed_data = pd.get_dummies(scaled_data, columns=discrete_features)

# Display the first few rows to verify the changes
preprocessed_data.head()



Skewness after transformation:
Age                               0.332082
original_shape_Elongation        -0.576687
original_shape_Flatness          -0.226737
original_shape_LeastAxisLength    0.354826
original_shape_MajorAxisLength    0.236364
                                    ...   
original_ngtdm_Busyness           0.144430
original_ngtdm_Coarseness         5.001462
original_ngtdm_Complexity         0.759852
original_ngtdm_Contrast           0.684131
original_ngtdm_Strength           1.319691
Length: 108, dtype: float64


C:\Users\Alvin\AppData\Local\Temp\ipykernel_47688\4189735657.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '135355.17109601933' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  series[index] = replacement_upper_limit


,Age,original_shape_Elongation,original_shape_Flatness,original_shape_LeastAxisLength,original_shape_MajorAxisLength,original_shape_Maximum2DDiameterColumn,original_shape_Maximum2DDiameterRow,original_shape_Maximum2DDiameterSlice,original_shape_Maximum3DDiameter,original_shape_MeshVolume,...,HistologyType_1.0,HistologyType_2.0,LNStatus_0.0,LNStatus_1.0,TumourStage_1.0,TumourStage_2.0,TumourStage_3.0,TumourStage_4.0,Gene_0.0,Gene_1.0
0,-0.442625,0.541324,0.480961,0.715195,0.217802,0.171980,0.681292,0.257650,0.317999,0.940230,...,True,False,False,True,False,True,False,False,False,True
1,-0.986501,0.802658,1.392499,0.257192,-0.592863,-0.433777,-0.174315,-0.075324,-0.408764,-0.099548,...,True,False,True,False,False,True,False,False,False,True
2,0.101252,1.116889,1.089542,0.434418,-0.309005,-0.106829,0.255037,0.005482,0.035263,0.327998,...,True,False,False,True,False,True,False,False,False,True
3,-0.533271,-0.044313,-0.910564,-0.266817,0.393816,0.427986,0.181208,0.716398,0.403574,0.115708,...,True,False,False,True,False,False,True,False,True,False
4,-1.167794,-0.366258,-0.613516,-1.059453,-0.508616,-0.910732,-0.464953,-0.416055,-0.719017,-0.937224,...,True,False,False,True,False,True,False,False,True,False


In [104]:
set(preprocessed_data.columns)

{'Age',
 'ChemoGrade_2.0',
 'ChemoGrade_3.0',
 'ER_0',
 'ER_1',
 'Gene_0.0',
 'Gene_1.0',
 'HER2_0',
 'HER2_1',
 'HistologyType_1.0',
 'HistologyType_2.0',
 'LNStatus_0.0',
 'LNStatus_1.0',
 'PgR_0.0',
 'PgR_1.0',
 'Proliferation_1.0',
 'Proliferation_2.0',
 'Proliferation_3.0',
 'TrippleNegative_0',
 'TrippleNegative_1',
 'TumourStage_1.0',
 'TumourStage_2.0',
 'TumourStage_3.0',
 'TumourStage_4.0',
 'original_firstorder_10Percentile',
 'original_firstorder_90Percentile',
 'original_firstorder_Energy',
 'original_firstorder_Entropy',
 'original_firstorder_InterquartileRange',
 'original_firstorder_Kurtosis',
 'original_firstorder_Maximum',
 'original_firstorder_Mean',
 'original_firstorder_MeanAbsoluteDeviation',
 'original_firstorder_Median',
 'original_firstorder_Minimum',
 'original_firstorder_Range',
 'original_firstorder_RobustMeanAbsoluteDeviation',
 'original_firstorder_RootMeanSquared',
 'original_firstorder_Skewness',
 'original_firstorder_TotalEnergy',
 'original_firstorder_

In [105]:
preprocessed_data = preprocessed_data.rename(columns={'HER2_1': 'HER2_1.0', 
                        'TumourStage_4.0': 'TumourStage_4', 
                        'TumourStage_3.0': 'TumourStage_3',
                        'TrippleNegative_1': 'TrippleNegative_1.0',
                        'TumourStage_2.0' : 'TumourStage_2'
                        })

In [106]:

X_selected_features = preprocessed_data[selected_features_b_pca]

In [107]:

categorical_vars = X_selected_features.columns[(X_selected_features.nunique() <= 5)]
continous_vars = X_selected_features.columns[(X_selected_features.nunique() > 5)]

X_continous = X_selected_features[continous_vars]

X_pca = pca.transform(X_continous)


In [ ]:
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()

# Step 4: Results
print("Explained Variance Ratio (each component):", explained_variance)
print("Cumulative Explained Variance:", cumulative_variance)

# Step 5: Create a DataFrame with Principal Components
pca_columns = [f'PC{i+1}' for i in range(X_pca.shape[1])]
X_pca_df = pd.DataFrame(X_pca, columns=pca_columns)

# Combine with the target variable if needed
X_test = pd.concat([X_pca_df, X_selected_features[categorical_vars].reset_index(drop=True)], axis=1)




Explained Variance Ratio (each component): [0.38700211 0.19631031 0.11665163 0.08725883 0.05864828 0.02976111
 0.02527514 0.02149344 0.01143027 0.00929064 0.00834147 0.00658536
 0.00531569 0.00451236 0.00438523 0.00366529 0.0027902  0.00234706
 0.00217366 0.00178405 0.00159154 0.00143441 0.00123674 0.00113313]
Cumulative Explained Variance: [0.38700211 0.58331243 0.69996406 0.78722289 0.84587117 0.87563228
 0.90090742 0.92240086 0.93383113 0.94312177 0.95146323 0.95804859
 0.96336428 0.96787664 0.97226187 0.97592716 0.97871736 0.98106442
 0.98323808 0.98502213 0.98661367 0.98804808 0.98928482 0.99041795]


In [109]:
# Predict on the test set
y_pred = best_model.predict(X_test)

# Print the predictions
print("Predictions on Test Set:", y_pred)


Predictions on Test Set: [53.252815 53.972454 54.335346 51.88235  51.70962  53.252815 53.73901
 53.13039  54.21292  53.252815 56.25964  54.21292  51.88235  53.85003
 53.972454 52.91572  53.13039  54.066387 53.79049  53.13039  53.13039
 52.134174 56.441605 54.335346 56.441605 56.406174 53.85003  56.465714
 54.15338  54.006847 53.972454 54.335346 51.70962  54.066387 51.88235
 54.15338  51.70962  51.88235  54.006847 53.85003  54.188812 54.335346
 51.88235  54.066387 54.188812 53.252815 54.335346 53.252815 56.465714
 51.88235  53.136166 54.335346 54.21292  54.188812 51.70962  54.335346
 54.335346 51.70962  53.252815 51.88235  51.88235  51.88235  53.13039
 51.70962  54.99657  51.88235  54.82384  51.88235  52.134174 51.70962
 54.335346 51.88235  51.88235  51.88235  54.066387 51.88235  51.88235
 54.99657  54.335346 54.99657  53.136166 54.335346 51.70962  51.88235
 51.903088 54.99657  51.88235  53.252815 51.88235  54.21292  51.88235
 51.88235  52.90272  54.99657  54.21292  54.335346 54.335346 

In [ ]:
y_pred.to_csv("RFSPrediction.csv", index=False)

AttributeError: 'numpy.ndarray' object has no attribute 'to_csv'

In [111]:
import pandas as pd

# Load the test dataset
data = pd.read_excel('FinalTestDataset2024.xls')

# Ensure y_pred is a DataFrame with a column for predictions
y_pred_df = pd.DataFrame(y_pred, columns=["Prediction"])

# Combine the existing 'ID' column with predictions
result = pd.concat([data['ID'], y_pred_df], axis=1)

# Save the result to a new CSV file
result.to_csv("RFSPrediction.csv", index=False)

print("Predictions saved with ID column in RFSPrediction.csv")


Predictions saved with ID column in RFSPrediction.csv


In [112]:
result

,ID,Prediction
0,TRG002219,53.252815
1,TRG002222,53.972454
2,TRG002223,54.335346
3,TRG002235,51.882351
4,TRG002240,51.709621
...,...,...
128,TRG002953,51.882351
129,TRG002955,51.882351
130,TRG002956,52.611492
131,TRG002957,52.954201
